In [2]:
import pandas as pd

# Import value labels and variable labels csvs
df_value_labels = pd.read_csv('value_labels.csv')
df_variable_labels = pd.read_csv('variable_labels.csv')

df_ivs_dictionary = pd.read_excel('F00011424-Common_EVS_WVS_Dictionary_IVS.xlsx', sheet_name='IVS_EVS_and_WVS_Variables')

df_countries_count = pd.read_csv('countries_count.csv')

# Merge value labels and variable labels csvs
df_labels = pd.merge(df_variable_labels, df_value_labels, left_on='vallab', right_on='lname', how='left')

# Create a label_lower value to remove duplicates
df_labels['label_lower'] = df_labels['label'].str.lower()

# Replace special characters in don't know labels
df_labels['label_lower'] = df_labels['label_lower'].str.replace("´", "'")

# Remove duplicates of label_lower by each name, vallab, varlab
df_labels = df_labels.drop_duplicates(subset=['name', 'vallab', 'varlab', 'label_lower'], keep='last')

# Create a value_label column with value and label
df_labels['value_label'] = df_labels['value'] + ': ' + df_labels['label']

# Combine values in value_label column by each name, vallab and varlab
df_labels = df_labels.groupby(['name', 'vallab', 'varlab'])['value_label'].apply(lambda x: ', '.join(x)).reset_index()

# Rename varlab to variable_label
df_labels = df_labels.rename(columns={'varlab': 'variable_label'})

# Select only the first three columns of the df_ivs_dictionary by their position
df_ivs_dictionary = df_ivs_dictionary.iloc[:, 1:4]
df_ivs_dictionary.columns = ['category', 'name', 'label']
df_ivs_dictionary = df_ivs_dictionary[['name', 'label', 'category']]

# Merge df_ivs_dictionary with df by name
df = pd.merge(df_ivs_dictionary, df_labels, on='name', how='left')

# Remove variables that are not questions
categories_not_for_questions = ["Admin/protocol variables", "Respondent's parents (EVS)", "Respondent's partner (EVS)", "Socio demographics"]

# Remove trailing spaces from category column
df['category'] = df['category'].str.strip()

df = df[~df['category'].isin(categories_not_for_questions)].reset_index(drop=True)

df = df[['name', 'variable_label', 'category', 'value_label']]

# Make the countries count data wide
df_countries_count_wide = df_countries_count.pivot(index='var', columns='year', values='count').reset_index()

# Merge the countries count data with the df
df = pd.merge(df, df_countries_count_wide, left_on='name', right_on='var', how='left')

# Remove the var column
df = df.drop(columns='var')

# Make numbers integers
df = df.fillna(0)
df = df.astype(int, errors='ignore')

# Save
df.to_csv('ivs_dictionary.csv', index=False)
